In [2]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

folder = 'fig'

## meanshift

In [6]:
cap = cv2.VideoCapture(Path(folder, 'Billard.mp4'))

if not cap.isOpened():
    print('video open failed!')
    sys.exit()

ret, back = cap.read()

(x, y, w, h) = cv2.selectROI(back)
rc = (x, y, w, h)
roi = back[y:y+h, x:x+w]
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)  # BGR => HSV

## 히스토그램
channels = [0, 1]
ranges = [0, 180, 0, 256]
hist = cv2.calcHist([roi_hsv], [0, 1], None, [90, 128], ranges)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproj = cv2.calcBackProject([frame_hsv], channels, hist, ranges, 1.)

    ## mean shift
    _, rc = cv2.meanShift(backproj, rc, term_crit)

    cv2.rectangle(frame, rc, (0, 0, 255), 2)

    cv2.imshow('frame', frame)

    if cv2.waitKey(30) == 27:
        break

cap.release()
cv2.destroyAllWindows()

## Motion vector

In [24]:
## Lucas-Canade

src1 = cv2.imread(Path(folder, 'frame1.jpg'))
src2 = cv2.imread(Path(folder, 'frame2.jpg'))

if src1 is None or src2 is None:
    print('Image read failed!')
    sys.exit()

gray1 = cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
pt1 = cv2.goodFeaturesToTrack(gray1, 60, 0.01, 10) #
pt2, status, err = cv2.calcOpticalFlowPyrLK(src1, src2, pt1, None)

dst = cv2.addWeighted(src1, 0.5, src2, 0.5, 0.0)

for i in range(pt2.shape[0]):
    if status[i, 0] == 0:
        continue
    
    srcPoints = (int(pt1[i, 0, 0]), int(pt1[i, 0, 1]))
    dstPoints = (int(pt2[i, 0, 0]), int(pt2[i, 0, 1]))
    cv2.circle(dst, srcPoints, 4, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.circle(dst, dstPoints, 4, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.arrowedLine(dst, srcPoints, dstPoints, (0, 255, 0), 1, cv2.LINE_AA)
    
# cv2.imshow('src1', src1)
# cv2.imshow('src2', src2)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()